In [3]:
import numpy as np
import pandas as pd
import sklearn
import re
import preprocessor as p
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm


In [2]:
!pip install tweet-preprocessor

  Using cached tweet_preprocessor-0.6.0-py3-none-any.whl (27 kB)


import dataset

In [4]:
# training data 
dataset_train = pd.read_csv('dataset-part2/train.csv')

# test data
dataset_test = pd.read_csv('dataset-part2/test.csv')

Data Explorasi

In [5]:
dataset_test.tail(2)

,id,tweet
17195,49158,"happy, at work conference: right mindset leads..."
17196,49159,"my song ""so glad"" free download! #shoegaze ..."


In [6]:
dataset_train.head(2)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...


In [7]:
# non-racist
sum(dataset_train['label']==0)

29720

In [9]:
# racist
sum(dataset_train['label']==1)

2242

In [10]:
#checking if there are missing value
dataset_train.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

Preprocesing


In [11]:
#install tweet-preprocessing for cleaning tweeets
WITH_SPACE = re.compile("(<br>\s/><br\s/?)|(-)|(/)|(:).")
NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")

In [12]:
# define function for cleaning the dataset (combining tweet_preprocessor and regex)
def clean_tweets(df):
    tempArr = []
    for line in df :
        #send to tweet_preprocessor
        tmpL = p.clean(line)
        tmpL = NO_SPACE.sub("", tmpL.lower()) #convert all tweets to lower case
        tmpL = WITH_SPACE.sub(" ", tmpL)
        tempArr.append(tmpL)
    return tempArr

In [17]:
# Clean training dataset
train = clean_tweets(dataset_train["tweet"])
train = pd.DataFrame(train)

In [18]:
# append cleaned tweets to the training data
dataset_train["clean_tweet"]= train
#than compare the cleaned and uncleaned tweets
dataset_train.head(10)

,id,label,tweet,clean_tweet
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for credit i cant use cause they dont o...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,i love u take with u all the time in ur
4,5,0,factsguide: society now #motivation,factsguide society now
5,6,0,[2/2] huge fan fare and big talking before the...,2 2 huge fan fare and big talking before they ...
6,7,0,@user camping tomorrow @user @user @user @use...,camping tomorrow danny
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams can...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land
9,10,0,@user @user welcome here ! i'm it's so #gr...,welcome here im its so


In [22]:
# clean the test data as like the train data
test_tweet = clean_tweets(dataset_test["tweet"])
test_tweet = pd.DataFrame(test_tweet)

# appending too
dataset_test["clean_tweet"] = test_tweet

#compare the cleaned again!
dataset_test.tail()

,id,tweet,clean_tweet
17192,49155,thought factory: left-right polarisation! #tru...,thought factory left right polarisation &gt3
17193,49156,feeling like a mermaid ð #hairflip #neverre...,feeling like a mermaid
17194,49157,#hillary #campaigned today in #ohio((omg)) &am...,today in omg &amp used words like assets&ampli...
17195,49158,"happy, at work conference: right mindset leads...",happy at work conference right mindset leads t...
17196,49159,"my song ""so glad"" free download! #shoegaze ...",my song so glad free download


TEST AND TRAIN SPLIT 


In [28]:
from sklearn.model_selection import train_test_split
# extract the labels from data
y = dataset_train.label.values

#use 80% for the traiining and 20% for testing
x_train, x_test, y_train, y_test = train_test_split(dataset_train.clean_tweet.values,
                                                     y,stratify=y,
                                                     random_state=1,
                                                     test_size=0.2,
                                                     shuffle=True) 

TO DO VECTORIZE TWEETS USING COUNTVECTORIZE

In [29]:
documents = ["This is Import Data's Youtube channel",
             "Data science is my passion and it is fun!",
             "Please subscribe to my channel"]

# initializing the countvectorize
vectorizer = CountVectorizer()

#tokenize and make the dociment into a matrix
document_into_matrix = vectorizer.fit_transform(documents)

#check the result 
pd.DataFrame(document_into_matrix.toarray(), columns= vectorizer.get_feature_names())

,and,channel,data,fun,import,is,it,my,passion,please,science,subscribe,this,to,youtube
0,0,1,1,0,1,1,0,0,0,0,0,0,1,0,1
1,1,0,1,1,0,2,1,1,1,0,1,0,0,0,0
2,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0


In [30]:
#vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')



In [31]:
#learn a vocabullary dictionary of all tokens in the raw document 
vectorizer.fit(list(x_train) + list(x_test))

# to do transform document to documnet_into_matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

CREATE MODEL 

In [35]:
from sklearn import svm
# classify using support vector machine
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data 
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_text
y_pred_svm = svm.predict(x_test_vec)

In [36]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  94.97888315344909 %
